In [1]:
#r "nuget:XPlot.Plotly,3.0.1"
#r "nuget:Microsoft.ML,1.5.2"
#r "D:\VSProjects\ITMO\ML\NonparametricRegression\bin\Release\netcoreapp3.1\NonparametricRegression.dll"
    
using XPlot.Plotly;
using NonparametricRegression;
using NonparametricRegression.Data;
using NonparametricRegression.Helpers;
using System.Linq;
using Microsoft.ML;
using System.Collections.Generic;
using System;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package XPlot.Plotly version 3.0.1

Installed package Microsoft.ML version 1.5.2

In [4]:
MLContext context = new MLContext();
IDataView data = context.Data.LoadFromTextFile<EcoliCsvData>("D:\\VSProjects\\ITMO\\ML\\ecoli.csv", ',', true);
var dataCsvRows = context
    .Data
    .CreateEnumerable<EcoliCsvData>(data, false)
    .ToList();

Int32[] classes = dataCsvRows
    .Select(k => k.Class)
    .Distinct()
    .ToArray();

var dict = Enumerable
    .Range(0, classes.Length)
    .ToDictionary(k => classes[k], e => e);
    
dataCsvRows.ForEach(k => k.Class = dict[k.Class]);
            
var dataRows = dataCsvRows
    .Select(k => k.ToDataSetObject())
    .ToList();

In [5]:
var matricesNaive = NonparametricRegression.Program.GetConfusionMatricesWithNaive(dataRows);
var plotMarker = new Graph.Marker() {size = 2};

var scattersNaive = new List<Graph.Scatter>
{
    new Graph.Scatter
    {
        name = "MicroF", marker = plotMarker, dx = 1, dy = 0.1,
        y = matricesNaive.Select(k => k.Item2.MicroF1Score(1)),
        x = matricesNaive.Select(k => k.Item1)
    },

    new Graph.Scatter
    {
         name = "MacroF", marker = plotMarker, dx = 1, dy = 0.1,
        y = matricesNaive.Select(k => k.Item2.MacroF1Score(1)),
        x = matricesNaive.Select(k => k.Item1)
    },
};
var chart = Chart.Plot(scattersNaive);
chart.WithXTitle("k-vlaue");
chart.WithYTitle("F value");
chart.WithTitle("Naive Tricube Manhattan");
display(chart);

In [6]:
var matricesOneHot = NonparametricRegression.Program.GetConfusionMatricesWithOneHot(dataRows);
var scattersOneHot = new List<Graph.Scatter>
{
    new Graph.Scatter
    {
        name = "MicroF", dx = 1, dy = 0.1, marker = plotMarker,
        y = matricesOneHot.Select(k => k.Item2.MicroF1Score(1)),
        x = matricesOneHot.Select(k => k.Item1)
    },

    new Graph.Scatter
    {
        name = "MacroF", dx = 1, dy = 0.1, marker = plotMarker,
        y = matricesOneHot.Select(k => k.Item2.MacroF1Score(1)),
        x = matricesOneHot.Select(k => k.Item1)
    },
};
var chart = Chart.Plot(scattersOneHot);
chart.WithXTitle("k-vlaue");
chart.WithYTitle("F value");
chart.WithTitle("OneHot Epanechnikov Euclidean");
display(chart);